In [9]:
import pandas  
import sklearn

In [ ]:
data = pandas.read_csv("movie_dataset.csv")
key_cols = ['title', 'genres', 'overview', 'keywords', 'runtime', 'budget', 'production_countries', 'production_companies']
data = data[key_cols]

In [17]:
genre_lists = data['genres'].fillna('').apply(
    lambda x: [g.strip() for g in x.split (',') if g.strip()]
)

country_lists = data['production_countries'].fillna('').apply(
    lambda x: [g.strip() for g in x.split (',') if g.strip()]
)

genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
genres_data = genres_encoder.fit_transform(genre_lists)

country_encoder = sklearn.preprocessing.MultiLabelBinarizer()
country_data = country_encoder.fit_transform(country_lists)

overview_encoder = sklearn.feature_extraction.text.TfidfVectorizer()
overview_data = overview_encoder.fit_transform(data['overview'].fillna(''))

keyword_encoder = sklearn.feature_extraction.text.TfidfVectorizer()
keyword_data = overview_encoder.fit_transform(data['keywords'].fillna(''))

numeric_data = data[['runtime', 'budget']].copy()

for col in ['runtime', 'budget']:
    median_val = numeric_data[col].median()
    numeric_data[col] = numeric_data[col].fillna(median_val)

scaler = sklearn.preprocessing.MinMaxScaler()
numeric_data_narmalized = scaler.fit_transform(numeric_data)

